## Falcon 40B Model Deployment

In [3]:
# install supported sagemaker SDK
!pip install "sagemaker==2.163.0" --upgrade --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
python-language-server 0.31.7 requires jedi<0.16,>=0.14.1, but you have jedi 0.18.2 which is incompatible.
python-language-server 0.31.7 requires ujson<=1.35; platform_system != "Windows", but you have ujson 5.7.0 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.
spyder 4.0.1 requires jedi==0.14.1, but you have jedi 0.18.2 which is incompatible.


In [22]:
import json
import sagemaker
import boto3
import os
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [23]:
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1' # ml.g5.xlarge, ml.g5.48xlarge
# os.environ['AWS_DEFAULT_REGION'] = 'us-west-2' # ml.g5.xlarge, ml.g5.2xlarge, ml.g5.4xlarge, ml.g5.8xlarge

In [24]:
region = boto3.Session().region_name
region

'us-east-1'

In [25]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'tiiuae/falcon-40b-instruct', 
    #'HF_MODEL_ID' : 'huggingface-llm-falcon-7b-instruct-bf16',
	'SM_NUM_GPUS': json.dumps(1)
}

endpoint_name = 'sm-contract-audit'

In [26]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

In [27]:
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.48xlarge",  
	container_startup_health_check_timeout=300,
    endpoint_name=endpoint_name
  )

--------------------*

UnexpectedStatusException: Error hosting endpoint sm-contract-audit: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint..

In [ ]:
predictor.predict({
	"inputs": "My name is Julien and I like to  ",
})

In [ ]:
def query_endpoint(payload):
    """Query endpoint and print the response"""
    response = predictor.predict(payload)
    #print(f"\033[1m Input:\033[0m {payload['inputs']}")
    print(f"\033[1m Output:\033[0m {response[0]['generated_text']}")

## Prompt Engineering

In [ ]:
code = """
 // SPDX-License-Identifier: MIT
 pragma solidity ^0.8.17;

 contract CrowdFunding {
     struct Campaign {
         address owner;
         string title;
         string description;
         uint256 target;
         uint256 deadline;
         uint256 amountCollected;
         string image;
         address[] donators;
         uint256[] donations;
     }

     mapping(uint256 => Campaign) public campaigns;

     uint256 public numberOfCampaigns = 0;

     function createCampaign(
         address _owner,
         string memory _title,
         string memory _description,
         uint256 _target,
         uint256 _deadline,
         string memory _image
     ) public returns (uint256) {
         Campaign storage campaign = campaigns[numberOfCampaigns];

         require(
             campaign.deadline < block.timestamp,
             "The deadline should be a date in the future."
         );

         campaign.owner = _owner;
         campaign.title = _title;
         campaign.description = _description;
         campaign.target = _target;
         campaign.deadline = _deadline;
         campaign.amountCollected = 0;
         campaign.image = _image;

         numberOfCampaigns++;

         return numberOfCampaigns - 1;
     }

     function donateToCampaign(uint256 _id) public payable {
         uint256 amount = msg.value;

         Campaign storage campaign = campaigns[_id];

         campaign.donators.push(msg.sender);
         campaign.donations.push(amount);

         campaign.amountCollected = campaign.amountCollected + amount;
     }

     function withdraw(uint256 _id) public {
         Campaign storage campaign = campaigns[_id];

         require(
             campaign.owner == msg.sender,
             "Only the owner of the campaign can withdraw the funds."
         );

         require(
             campaign.amountCollected >= campaign.target,
             "The campaign has not reached its target yet."
         );

         require(
             campaign.deadline < block.timestamp,
             "The deadline has not passed yet."
         );

         (bool sent, ) = payable(campaign.owner).call{
             value: campaign.amountCollected
         }("");

         if (sent) {
             campaign.amountCollected = 0;
         }
     }

     function getCampaign(uint256 id)
         public
         view
         returns (
             address,
             string memory,
             string memory,
             uint256,
             uint256,
             uint256,
             string memory
         )
     {
         Campaign memory campaign = campaigns[id];
         return (
             campaign.owner,
             campaign.title,
             campaign.description,
             campaign.target,
             campaign.deadline,
             campaign.amountCollected,
             campaign.image
         );
     }
 }
"""

In [ ]:
# define payload
prompt = f"""You are an helpful Assistant, called Falcon. Knowing everyting about Smart Conctracts.

User: Can you list the security vulnerability in the following smart contract code?

{code}

Falcon:"""

# hyperparameters for llm
payload = {
  "inputs": prompt,
  "parameters": {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.8,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.03,
    "stop": ["\nUser:","<|endoftext|>","</s>"]
  }
}

# send request to endpoint
response = predictor.predict(payload)

# print assistant respond
assistant = response[0]["generated_text"][len(prompt):]
print(assistant)

In [ ]:
# query_endpoint(payload)

In [ ]:
# new_prompt = f"""{prompt}{assistant}
# User: How would you recommend start using Amazon SageMaker? If i am new to Machine Learning?
# Falcon:"""
# # update payload
# payload["inputs"] = new_prompt

# # send request to endpoint
# response = predictor.predict(payload)

# # print assistant respond
# new_assistant = response[0]["generated_text"][len(new_prompt):]
# print(new_assistant)

In [ ]:
# query_endpoint(payload):

In [ ]:
## Delete Endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()